In [1]:
import pathlib

import numpy as np
import pandas as pd

In [2]:
# set the path to the data
academic_data_path = pathlib.Path(
    "../../data/All_academic_projects_funded_by_NIH_.xlsx"
).resolve(strict=True)
output_data_path = pathlib.Path(
    "../../data/All_academic_projects_funded_by_NIH_cleaned.parquet"
).resolve()
# get the sheet names in the excel file
academic_data = pd.read_excel(academic_data_path, sheet_name="#205C", skiprows=2)
print(academic_data.shape)
academic_data.head()

(1004, 7)


/home/lippincm/miniforge3/envs/NIH_env/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,Institute/Center,Mechanism/Funding Source,Activity Code,Number of Applications Reviewed,Number of Applications Awarded,Success Rate,Total Funding
0,NCI,RPG - Direct,P01,89,17,0.191011,41733718
1,NCI,RPG - Direct,R00,1,1,1.000000,783430
2,NCI,RPG - Direct,R01,5442,793,0.145718,437729824
3,NCI,RPG - Direct,R03,369,50,0.135501,4817701
4,NCI,RPG - Direct,R15,138,20,0.144928,8818798


In [3]:
# drop activity codes that equal to 'Total'
academic_data = academic_data[academic_data["Activity Code"] != "Total"]
print(academic_data.shape)

(909, 7)


In [4]:
# move the Activity Code to the first column
academic_data.insert(0, "Activity Code", academic_data.pop("Activity Code"))
academic_data.head()

,Activity Code,Institute/Center,Mechanism/Funding Source,Number of Applications Reviewed,Number of Applications Awarded,Success Rate,Total Funding
0,P01,NCI,RPG - Direct,89,17,0.191011,41733718
1,R00,NCI,RPG - Direct,1,1,1.000000,783430
2,R01,NCI,RPG - Direct,5442,793,0.145718,437729824
3,R03,NCI,RPG - Direct,369,50,0.135501,4817701
4,R15,NCI,RPG - Direct,138,20,0.144928,8818798


In [5]:
academic_data["Mechanism/Funding Source"].value_counts()

Mechanism/Funding Source
Other Mechanisms - Direct                                         594
RPG - Direct                                                      282
RPG - Reimbursable                                                 12
Other Mechanisms - Reimbursable                                     7
Direct Authorization - Type 1 Diabetes Other Activities             5
Other Mechanisms - Superfund                                        4
Direct Authorization - Type 1 Diabetes Research Project Grants      3
RPG - Superfund                                                     2
Name: count, dtype: int64

In [6]:
# keep only the Other Mechanisms - Direct and RPG - Direct for mechanisms
academic_data = academic_data[
    academic_data["Mechanism/Funding Source"].isin(
        ["Other Mechanisms - Direct", "RPG - Direct"]
    )
]
print(academic_data.shape)
academic_data.head()

(876, 7)


,Activity Code,Institute/Center,Mechanism/Funding Source,Number of Applications Reviewed,Number of Applications Awarded,Success Rate,Total Funding
0,P01,NCI,RPG - Direct,89,17,0.191011,41733718
1,R00,NCI,RPG - Direct,1,1,1.000000,783430
2,R01,NCI,RPG - Direct,5442,793,0.145718,437729824
3,R03,NCI,RPG - Direct,369,50,0.135501,4817701
4,R15,NCI,RPG - Direct,138,20,0.144928,8818798


In [7]:
# remove OD COMMON FUND, FIC, OD ORIP
academic_data = academic_data[
    ~academic_data["Institute/Center"].isin(["OD COMMON FUND", "FIC", "OD ORIP"])
]
print(academic_data.shape)
academic_data.head()

(828, 7)


,Activity Code,Institute/Center,Mechanism/Funding Source,Number of Applications Reviewed,Number of Applications Awarded,Success Rate,Total Funding
0,P01,NCI,RPG - Direct,89,17,0.191011,41733718
1,R00,NCI,RPG - Direct,1,1,1.000000,783430
2,R01,NCI,RPG - Direct,5442,793,0.145718,437729824
3,R03,NCI,RPG - Direct,369,50,0.135501,4817701
4,R15,NCI,RPG - Direct,138,20,0.144928,8818798


In [8]:
# write the cleaned data to a parquet file
academic_data.to_parquet(output_data_path, index=False)